In [2]:
import gym
env = gym.make("MountainCar-v0").env

In [3]:
episodes = 50
penalties = 0
done = False

In [4]:
#iperparametri
alfa = 0.08
gamma = 0.6
epsilon = 0.1

In [5]:
#inizializzazione della q_table con tutti elementi nulli
import numpy as np
import random
num_states = (env.observation_space.high - env.observation_space.low)*np.array([10, 100])
num_states = np.round(num_states, 0).astype(int) + 1
q_table = np.zeros([num_states[0], num_states[1], env.action_space.n])

In [6]:
#addestramento della q_table
total_iterations = []
for i in range(episodes):
    observation = env.reset()
    state = (observation - env.observation_space.low)*np.array([10, 100])
    state = np.round(state, 0).astype(int)
    reward = 0
    iteration = 0
    done = False
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state[0],state[1]])
        new_observation, reward, done, info = env.step(action)
        new_state = (new_observation - env.observation_space.low)*np.array([10, 100])
        new_state = np.round(new_state, 0).astype(int)

        reward = reward * (0.5 - new_observation[0]) * 10

        old_q = q_table[state[0],state[1], action]

        q_table[state[0],state[1], action] = old_q * (1-alfa) + alfa * (reward + gamma * np.max(q_table[new_state[0],new_state[1]]))

        iteration = iteration + 1
        env.render(mode='close')
        state = new_state
    total_iterations.append(iteration)
    print("Episode {} terminated!".format(i))


Episode 0 terminated!
Episode 1 terminated!
Episode 2 terminated!
Episode 3 terminated!
Episode 4 terminated!
Episode 5 terminated!
Episode 6 terminated!
Episode 7 terminated!
Episode 8 terminated!
Episode 9 terminated!
Episode 10 terminated!
Episode 11 terminated!
Episode 12 terminated!
Episode 13 terminated!
Episode 14 terminated!
Episode 15 terminated!
Episode 16 terminated!
Episode 17 terminated!
Episode 18 terminated!
Episode 19 terminated!
Episode 20 terminated!
Episode 21 terminated!
Episode 22 terminated!
Episode 23 terminated!
Episode 24 terminated!
Episode 25 terminated!
Episode 26 terminated!
Episode 27 terminated!
Episode 28 terminated!
Episode 29 terminated!
Episode 30 terminated!
Episode 31 terminated!
Episode 32 terminated!
Episode 33 terminated!
Episode 34 terminated!
Episode 35 terminated!
Episode 36 terminated!
Episode 37 terminated!
Episode 38 terminated!
Episode 39 terminated!
Episode 40 terminated!
Episode 41 terminated!
Episode 42 terminated!
Episode 43 terminated

In [7]:
q_table

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-6.22112249e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.95022980e+01, -3.04280919e+01, -2.95888843e+01],
        [-3.70152589e+01, -3.72126376e+01, -3.70132664e+01],
        [-3.50947035e+01, -3.53082247e+01, -3.54173174e+01],
        [-3.57393588e+01, -3.61481194e+01, -3.59287037e+01],
        [-4.06728272e+01, -4.06086835e+01, -4.06670461e+01],
        [-3.95812806e+01, -3.95400139e+01, -3.93839278e+01],
        [-3.06436862e+01, -3.03044099e+01, -3.02794859e+01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+

In [11]:
#Test dell'agente
total_epochs, total_penalties = 0, 0
episodes = 2

for _ in range(episodes):
    observation = env.reset()
    state = (observation - env.observation_space.low)*np.array([10, 100])
    state = np.round(state, 0).astype(int)
    epochs, penalties, reward = 0,0,0

    done = False
    i=0
    while not done:
        action = np.argmax(q_table[state[0],state[1]])
        new_observation, reward, done, info = env.step(action)
        state = (new_observation - env.observation_space.low)*np.array([10, 100])
        state = np.round(state, 0).astype(int)
        epochs += 1
        env.render()

    total_epochs += epochs

In [12]:
print("Iterazioni complessive {} per un numero di episodi di {}.".format(total_epochs, episodes))

Iterazioni complessive 602 per un numero di episodi di 2.


In [19]:
#Salva la Q-table
np.save("q_table", q_table)

In [18]:
#Caricare una Q-table nuova
q_table = np.load("q_table.npy")
